### Install all Dependencies

In [ ]:
!pip install matplotlib
!pip install tensorflow 2.4.1





In [ ]:
from tensorflow.keras import backend
backend.clear_session()

### Import Dependencies

In [ ]:
# Import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# Import tensorflow dependencies - Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

### Set GPU Growth

In [ ]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimentl_set_memory_growth(gpu , True)

### Create Folder Structures

In [ ]:
POS_PATH = os.path.join("data" ,"positive")
NEG_PATH = os.path.join("data" , "negitive")
ANC_PATH = os.path.join("data" , "anchor")

### Collect Positives and Anchors
### Untar Labelled Faces in the Wild Dataset

In [ ]:
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

In [ ]:
# Uncompress Tar GZ Labelled Faces in the Wild Dataset
!tar -xf lfw.tgz


In [ ]:
for directory in os.listdir("lfw"):
    for file in os.listdir(os.path.join("lfw", directory)):
        EX_PATH = os.path.join("lfw" , directory , file)
        NEW_PATH = os.path.join(NEG_PATH , file)
        os.replace(EX_PATH , NEW_PATH)

### Collect Positive and Anchor Classes

In [ ]:
 #Import uuid library to generate unique image names
import uuid


In [ ]:
os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))


In [ ]:


# Establish a connection to the webcam
cap = cv2.VideoCapture(0)
while cap.isOpened(): 
    ret, frame = cap.read()
   
    # Cut down frame to 250x250px
    frame = frame[120:120+250,200:200+250, :]
    
    # Collect anchors 
    if cv2.waitKey(1) & 0XFF == ord('a'):
        # Create the unique file path 
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out anchor image
        cv2.imwrite(imgname, frame)
    
    # Collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Create the unique file path 
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        # Write out positive image
        cv2.imwrite(imgname, frame)
    
    # Show image back to screen
    cv2.imshow('Image Collection', frame)
    
    # Breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    
    ret , frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    # Cut down frame to 250x250px
   
   #frame = frame[120:120+250,200:200+250, :]
    cv2.imshow("Imgage collection" , frame)
    if cv2.waitKey(1) & 0XFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
plt.imshow(frame)

In [ ]:
plt.imshow(frame[120:120+250,200:200+300, :])

In [ ]:
anchor = tf.data.Dataset.list_files(ANC_PATH+'\*.jpg').take(250)
positive = tf.data.Dataset.list_files(POS_PATH+'\*.jpg').take(250)
negative = tf.data.Dataset.list_files(NEG_PATH+'\*.jpg').take(250)

In [ ]:
dir_test = anchor.as_numpy_iterator()


In [ ]:
print(dir_test.next())

### Data Augmentation

In [ ]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 100x100x3
    img = tf.image.resize(img, (100,100))
    # Scale image to be between 0 and 1 
    img = img / 255.0
    
    # Return image
    return img

In [ ]:
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img , (100 ,100))
    img = img/255.0
    return img

In [ ]:
img = preprocess('data\\anchor\\593bc1db-66f5-11ec-9c63-107b44d9d44c.jpg')

In [ ]:
plt.imshow(img)

In [ ]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [ ]:
def preprocess_twin(input_img , validation_img , label):
    return(preprocess(input_img), preprocess(validation_img) , label)

In [ ]:
samples = data.as_numpy_iterator()
exampple = samples.next()
exampple

In [ ]:
res = preprocess_twin(*exampple)

In [ ]:
plt.imshow(res[1])


In [ ]:
res[2]

In [ ]:
# build dataloader pipline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size = 1024)

In [ ]:
samples = data.as_numpy_iterator()
samples.next()

In [ ]:
samp = samples.next()

In [ ]:
plt.imshow(samp[0])

In [ ]:
plt.imshow(samp[1])

In [ ]:
samp[2]

In [ ]:
# training data portion

train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)





In [ ]:
# Testing partition
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

### Model Engineering
### Build Embedding Layer

In [ ]:
def make_embedding():
    inp = Input(shape = (100,100,3), name = "input_shape")
    # first block 
    c1 = Conv2D(64 ,(10,10) ,activation = "relu" )(inp)
    m1 = MaxPooling2D(64,(2,2) , padding = "same")(c1)
    # second block 
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)
    
    # third block
    c3 = Conv2D(128 , (4,4), activation = "relu")(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)
    
    # final embedding layer 
    c4 = Conv2D(256 , (4,4) , activation = "relu")(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096 , activation = "sigmoid")(f1)
    
    return Model(inputs=[inp] , outputs=[d1],name = "embedding")

In [ ]:
embedding = make_embedding()


In [ ]:
embedding.summary()

### Build Distance Layer

In [ ]:
# Siamese L1 Distance class
class L1Dist(Layer):
    
    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()
       
    # Magic happens here - similarity calculation
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [ ]:
l1 = L1Dist()

In [ ]:
l1(anchor_embedding, validation_embedding)

### Make Siamese Model

In [ ]:
def make_siamese_model(): 
    
    # Anchor image input in the network
    input_image = Input(name='input_img', shape=(100,100,3))
    
    # Validation image in the network 
    validation_image = Input(name='validation_img', shape=(100,100,3))
    
    # Combine siamese distance components
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    
    # Classification layer 
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='SiameseNetwork')

In [ ]:
siamese_model = make_siamese_model()

In [ ]:
siamese_model.summary()

### Training
### Setup Loss and Optimizer

In [ ]:
binary_cross_loss = tf.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.Adam(1e-4) # 0.0001

### Establish Checkpoints

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

### Build Train Step Function

In [ ]:
@tf.function
def train_step(batch):
    
    # Record all of our operations 
    with tf.GradientTape() as tape:     
        # Get anchor and positive/negative image
        X = batch[:2]
        # Get label
        y = batch[2]
        
        # Forward pass
        yhat = siamese_model(X, training=True)
        # Calculate loss
        loss = binary_cross_loss(y, yhat)
    print(loss)
        
    # Calculate gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    
    # Calculate updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
    
    # Return loss
    return loss

### Build Training Loop

In [ ]:
def train(data, EPOCHS):
    # Loop through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        # Loop through each batch
        for idx, batch in enumerate(data):
            # Run train step here
            train_step(batch)
            progbar.update(idx+1)
        
        # Save checkpoints
        if epoch % 5 == 0: 
            checkpoint.save(file_prefix=checkpoint_prefix)

### Train the model

In [ ]:
EPOCHS = 50

In [ ]:
train(train_data, EPOCHS)

### Evaluate Model
### Import Metrics

In [ ]:
# Import metric calculations
from tensorflow.keras.metrics import Precision, Recall

In [ ]:
# Get a batch of test data
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [ ]:
# Make predictions
y_hat = siamese_model.predict([test_input, test_val])
y_hat

In [ ]:
# Post processing the results 
[1 if prediction > 0.5 else 0 for prediction in y_hat ]


In [ ]:
y_true

### Calculate Metrics

In [ ]:
# Creating a metric object 
m = Recall()

# Calculating the recall value 
m.update_state(y_true, y_hat)

# Return Recall Result
m.result().numpy()

In [ ]:
# Creating a metric object 
m = Precision()

# Calculating the recall value 
m.update_state(y_true, y_hat)

# Return Recall Result
m.result().numpy()

### Viz Results

In [ ]:
# Set plot size 
plt.figure(figsize=(10,8))

# Set first subplot
plt.subplot(1,2,1)
plt.imshow(test_input[0])

# Set second subplot
plt.subplot(1,2,2)
plt.imshow(test_val[0])

# Renders cleanly
plt.show()

## Save Model

In [ ]:
# Save weights
siamese_model.save('siamesemodel.h5')

In [ ]:
# Reload model 
model = tf.keras.models.load_model('siamesemodel.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [ ]:
# Make predictions with reloaded model
model.predict([test_input, test_val])

In [ ]:
# View model summary
model.summary()

### Real Time Test
### Verification Function

In [ ]:
application_data\verification_images

In [ ]:
os.listdir(os.path.join('application_data', 'verification_images'))

In [ ]:
os.path.join('application_data', 'input_image', 'input_image.jpg')

In [ ]:
def verify(model, detection_threshold, verification_threshold):
    # Build results array
    results = []
    for image in os.listdir(os.path.join('application_data', 'verification_images')):
        input_img = preprocess(os.path.join('application_data', 'input_image', 'input_image.jpg'))
        validation_img = preprocess(os.path.join('application_data', 'verification_images', image))
        
        # Make Predictions 
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
    
    # Detection Threshold: Metric above which a prediciton is considered positive 
    detection = np.sum(np.array(results) > detection_threshold)
    
    # Verification Threshold: Proportion of positive predictions / total positive samples 
    verification = detection / len(os.listdir(os.path.join('application_data', 'verification_images'))) 
    verified = verification > verification_threshold
    
    return results, verified

In [ ]:
def verify(model . detection_threshold , verification_threshold):
    ## build the result 
    results = []
    for image in os.listdir(os.path.join("application_data" , "validation_images")):
        input_img = preprocess(os.path.join("appliction_data" , "input_image" , "input_image.jpg"))
        vslidation_img = preprocess(os.path.join("application_data" , "verification_images" . image))
        
        
        # make_pediction
        
        result = model.predict(list(np.expand_dims([input_img , validation_img], axis = 1)))
        
        results.append(result)
        
    detection = no.sum(np.array(results)> detection_threshold)
    
    verificaion = detection / len(os.listdir(os.apth.join("application_data" , "verfication_images")))
    verified = verificaion > verification_threshold
    
    return result

### OpenCV Real Time Verification

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    
    cv2.imshow('Verification', frame)
    
    # Verification trigger
    if cv2.waitKey(10) & 0xFF == ord('v'):
        # Save input image to application_data/input_image folder 
        cv2.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
        # Run verification
        results, verified = verify(model, 0.9, 0.7)
        print(verified)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
np.sum(np.squeeze(results) > 0.9)

In [ ]:
results